# 1. Neural networks - MLP and DNN

Multi-layer Perceptron (MLP) is a supervised learning algorithm that learns a function by training on a dataset. It is a universal approximator of any function. General work principles rely on feedforward signal propagation to predict and backward error propagation to adjust weights and adapt to problems/data.

\begin{equation*}
output = f(w_0 + w_1 * x_1 + ... + w_i * x_i)
\end{equation*}

![Perceptron.png](./img/Perceptron.png)

Source: https://pl.wikipedia.org/wiki/Perceptron#/media/Plik:Perceptron_moj.png

![ANN2.png](./img/ANN2.png)

Source: https://en.wikipedia.org/wiki/Neural_network

A deep neural network (DNN) is an artificial neural network (ANN) with more complex architecture within a number of layers. The DNN during a learning process adapts to data and gathered knowledge is stored in weights conecting artificial neurons. Development and learning of a very complex neural network require a specific approach called deep learning which is related to technics preventing overfitting as well as reduce the time required to learn the model. DNN could have some specilized layers introduced like filters responsible for data convolution (Convolutional Neural Networks).


# 2. Problem - predict if given chemical compound is agonist of estrogen nuclear receptor alpha

The 2014 Tox21 data challenge was designed to develop models capable to predict the affinity of chemical compounds to specific receptors. General idea was to develop a tools with possible applications in searching for new active compounds and perform screening in search of new potential drugs. We will focus on the assessment of compound affinity to estrogen alpha receptors. 

qHTS assay to identify small molecule agonists of the estrogen receptor alpha (ER-alpha) signaling pathway using the BG1 cell line

Tox21 challange: https://tripod.nih.gov/tox21/challenge/index.jsp

Dataset we will explore: https://pubchem.ncbi.nlm.nih.gov/bioassay/743079

![data.png](./img/data.png)

# 3. Packages we will use and explore

RDKit is Open source toolkit for cheminformatics which enable operations of moleculs representation to calculate 2D and 3D descriptors as well as fingerprints
https://www.rdkit.org/docs/index.html

NumPy provides numerous ready to use solutions for performing numerical operations such as exponentiation, logarithmic, etc.
https://numpy.org/

Pandas provide a number of ready to use solutions for data analysis, including ready structures for data storage, quick statistical summary, record deletion.
https://pandas.pydata.org/

Scikit-learn provides a number of tools for data manipulation and data pre-processing as well as predictive models development, and evaluation.
https://scikit-learn.org/stable/ 

Keras is an open-source neural network library written in Python. It is capable of running on top of TensorFlow, Microsoft Cognitive Toolkit, R, Theano, or PlaidML.Designed to enable fast experimentation with deep neural networks.
https://keras.io/

In [1]:
##Import all required packages
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import *
from keras.wrappers.scikit_learn import *
#Load the data
df_raw = pd.read_csv("estrogen_nuclear_receptor.txt", sep="\t")

#Function to calculate fingerprints with specified length of binary outcome
def morgan_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=10000)
    npfp=np.array(list(fp.ToBitString())).astype('float').T
    return npfp

Using TensorFlow backend.


Let's calculate fingerprints for molecules - to find more about usage of fingerprints in drug development you may find this article useful: 
https://www.sciencedirect.com/science/article/pii/S1046202314002631#f0010

In [2]:
data=np.array([np.arange(df_raw.shape[0])]*10000).T
for i, row in df_raw.iterrows():
    data[i]=morgan_fp(df_raw["SMILES"][i])

In [3]:
data.shape

(8472, 10000)

In [4]:
#Create input and output vectors of variables
X=data
y=pd.DataFrame(df_raw['Agonist'])

In [23]:
#Define model architecture and perform 10-cv calculations
def DNN_model():
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_initializer='normal', activation='relu',    kernel_regularizer=l1_l2(l1=0.0, l2=0.05)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_initializer='normal', activation='relu', kernel_regularizer=l1_l2(l1=0.0, l2=0.05)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_initializer='normal', activation='relu', kernel_regularizer=l1_l2(l1=0.0, l2=0.05)))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
    return model

dnn = KerasClassifier(build_fn=DNN_model, epochs=100, batch_size=1000, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
results = cross_validate(dnn, 
                        X=X, 
                        y=y, 
                        cv=kfold, 
                        scoring=['accuracy', 'balanced_accuracy', 'roc_auc', 'f1'], 
                        return_train_score=True, 
                        return_estimator=True)


Epoch 1/100
7624/7624 [==============================] - 3s 437us/step - loss: 4.5696 - accuracy: 0.8804
Epoch 2/100
7624/7624 [==============================] - 1s 182us/step - loss: 3.4001 - accuracy: 0.8804
Epoch 3/100
7624/7624 [==============================] - 1s 161us/step - loss: 2.8504 - accuracy: 0.8804
Epoch 4/100
7624/7624 [==============================] - 1s 173us/step - loss: 2.3846 - accuracy: 0.8804
Epoch 5/100
7624/7624 [==============================] - 1s 163us/step - loss: 1.9878 - accuracy: 0.8804
Epoch 6/100
7624/7624 [==============================] - 1s 161us/step - loss: 1.6697 - accuracy: 0.8805
Epoch 7/100
7624/7624 [==============================] - 1s 148us/step - loss: 1.4144 - accuracy: 0.8804
Epoch 8/100
7624/7624 [==============================] - 1s 146us/step - loss: 1.2056 - accuracy: 0.8808
Epoch 9/100
7624/7624 [==============================] - 1s 154us/step - loss: 1.0379 - accuracy: 0.8813
Epoch 10/100
7624/7624 [==============================]

7624/7624 [==============================] - 1s 159us/step - loss: 0.1132 - accuracy: 0.9425
Epoch 79/100
7624/7624 [==============================] - 1s 195us/step - loss: 0.1146 - accuracy: 0.9389
Epoch 80/100
7624/7624 [==============================] - 1s 193us/step - loss: 0.1126 - accuracy: 0.9432
Epoch 81/100
7624/7624 [==============================] - 1s 178us/step - loss: 0.1171 - accuracy: 0.9374
Epoch 82/100
7624/7624 [==============================] - 1s 165us/step - loss: 0.1129 - accuracy: 0.9424
Epoch 83/100
7624/7624 [==============================] - 1s 183us/step - loss: 0.1123 - accuracy: 0.9418
Epoch 84/100
7624/7624 [==============================] - 1s 196us/step - loss: 0.1133 - accuracy: 0.9387
Epoch 85/100
7624/7624 [==============================] - 2s 254us/step - loss: 0.1107 - accuracy: 0.9420
Epoch 86/100
7624/7624 [==============================] - 1s 194us/step - loss: 0.1086 - accuracy: 0.9450
Epoch 87/100
7624/7624 [==============================] - 2

7624/7624 [==============================] - 1s 178us/step - loss: 0.1247 - accuracy: 0.9419
Epoch 54/100
7624/7624 [==============================] - 1s 182us/step - loss: 0.1234 - accuracy: 0.9427
Epoch 55/100
7624/7624 [==============================] - 1s 160us/step - loss: 0.1249 - accuracy: 0.9466
Epoch 56/100
7624/7624 [==============================] - 1s 157us/step - loss: 0.1327 - accuracy: 0.9398
Epoch 57/100
7624/7624 [==============================] - 1s 182us/step - loss: 0.1318 - accuracy: 0.9415
Epoch 58/100
7624/7624 [==============================] - 1s 177us/step - loss: 0.1317 - accuracy: 0.9457
Epoch 59/100
7624/7624 [==============================] - 1s 196us/step - loss: 0.1273 - accuracy: 0.9448
Epoch 60/100
7624/7624 [==============================] - 1s 184us/step - loss: 0.1302 - accuracy: 0.9382
Epoch 61/100
7624/7624 [==============================] - 1s 168us/step - loss: 0.1260 - accuracy: 0.9437
Epoch 62/100
7624/7624 [==============================] - 1

7625/7625 [==============================] - 1s 172us/step - loss: 0.2983 - accuracy: 0.9390
Epoch 29/100
7625/7625 [==============================] - 1s 157us/step - loss: 0.2909 - accuracy: 0.9440
Epoch 30/100
7625/7625 [==============================] - 1s 166us/step - loss: 0.2826 - accuracy: 0.9420
Epoch 31/100
7625/7625 [==============================] - 1s 180us/step - loss: 0.2740 - accuracy: 0.9422
Epoch 32/100
7625/7625 [==============================] - 1s 168us/step - loss: 0.2624 - accuracy: 0.9419
Epoch 33/100
7625/7625 [==============================] - 1s 164us/step - loss: 0.2625 - accuracy: 0.9436
Epoch 34/100
7625/7625 [==============================] - 1s 174us/step - loss: 0.2532 - accuracy: 0.9388
Epoch 35/100
7625/7625 [==============================] - 1s 172us/step - loss: 0.2514 - accuracy: 0.9435
Epoch 36/100
7625/7625 [==============================] - 1s 166us/step - loss: 0.2430 - accuracy: 0.9462
Epoch 37/100
7625/7625 [==============================] - 1

7625/7625 [==============================] - 1s 178us/step - loss: 2.5115 - accuracy: 0.8803
Epoch 4/100
7625/7625 [==============================] - 1s 193us/step - loss: 2.0031 - accuracy: 0.8805
Epoch 5/100
7625/7625 [==============================] - 1s 193us/step - loss: 1.6112 - accuracy: 0.8807
Epoch 6/100
7625/7625 [==============================] - 1s 175us/step - loss: 1.3082 - accuracy: 0.8810
Epoch 7/100
7625/7625 [==============================] - 1s 180us/step - loss: 1.0703 - accuracy: 0.8828
Epoch 8/100
7625/7625 [==============================] - 1s 171us/step - loss: 0.8877 - accuracy: 0.8858
Epoch 9/100
7625/7625 [==============================] - 1s 160us/step - loss: 0.7385 - accuracy: 0.8911
Epoch 10/100
7625/7625 [==============================] - 1s 155us/step - loss: 0.6193 - accuracy: 0.9002
Epoch 11/100
7625/7625 [==============================] - 1s 184us/step - loss: 0.5269 - accuracy: 0.9062
Epoch 12/100
7625/7625 [==============================] - 1s 153u

...

7625/7625 [==============================] - 1s 169us/step - loss: 0.1267 - accuracy: 0.9411
Epoch 83/100
7625/7625 [==============================] - 1s 163us/step - loss: 0.1295 - accuracy: 0.9390
Epoch 84/100
7625/7625 [==============================] - 1s 165us/step - loss: 0.1261 - accuracy: 0.9343
Epoch 85/100
7625/7625 [==============================] - 1s 180us/step - loss: 0.1238 - accuracy: 0.9398
Epoch 86/100
7625/7625 [==============================] - 1s 156us/step - loss: 0.1276 - accuracy: 0.9369
Epoch 87/100
7625/7625 [==============================] - 1s 179us/step - loss: 0.1262 - accuracy: 0.9388
Epoch 88/100
7625/7625 [==============================] - 2s 197us/step - loss: 0.1229 - accuracy: 0.9411
Epoch 89/100
7625/7625 [==============================] - 1s 171us/step - loss: 0.1201 - accuracy: 0.9364
Epoch 90/100
7625/7625 [==============================] - 1s 174us/step - loss: 0.1221 - accuracy: 0.9410
Epoch 91/100
7625/7625 [==============================] - 1

7625/7625 [==============================] - 2s 215us/step - loss: 0.1400 - accuracy: 0.9447
Epoch 58/100
7625/7625 [==============================] - 2s 257us/step - loss: 0.1360 - accuracy: 0.9422
Epoch 59/100
7625/7625 [==============================] - 2s 206us/step - loss: 0.1388 - accuracy: 0.9433
Epoch 60/100
7625/7625 [==============================] - 2s 228us/step - loss: 0.1338 - accuracy: 0.9415
Epoch 61/100
7625/7625 [==============================] - 2s 209us/step - loss: 0.1419 - accuracy: 0.9406
Epoch 62/100
7625/7625 [==============================] - 1s 169us/step - loss: 0.1375 - accuracy: 0.9450
Epoch 63/100
7625/7625 [==============================] - 1s 165us/step - loss: 0.1338 - accuracy: 0.9402
Epoch 64/100
7625/7625 [==============================] - 1s 166us/step - loss: 0.1322 - accuracy: 0.9415
Epoch 65/100
7625/7625 [==============================] - 1s 148us/step - loss: 0.1262 - accuracy: 0.9450
Epoch 66/100
7625/7625 [==============================] - 1

7625/7625 [==============================] - 1s 188us/step - loss: 0.1837 - accuracy: 0.9411
Epoch 33/100
7625/7625 [==============================] - 1s 166us/step - loss: 0.1819 - accuracy: 0.9432
Epoch 34/100
7625/7625 [==============================] - 1s 146us/step - loss: 0.1785 - accuracy: 0.9426
Epoch 35/100
7625/7625 [==============================] - 1s 159us/step - loss: 0.1687 - accuracy: 0.9419
Epoch 36/100
7625/7625 [==============================] - 1s 169us/step - loss: 0.1757 - accuracy: 0.9401
Epoch 37/100
7625/7625 [==============================] - 1s 177us/step - loss: 0.1625 - accuracy: 0.9435
Epoch 38/100
7625/7625 [==============================] - 1s 159us/step - loss: 0.1646 - accuracy: 0.9428
Epoch 39/100
7625/7625 [==============================] - 1s 169us/step - loss: 0.1573 - accuracy: 0.9443
Epoch 40/100
7625/7625 [==============================] - 2s 207us/step - loss: 0.1638 - accuracy: 0.9424
Epoch 41/100
7625/7625 [==============================] - 2

7625/7625 [==============================] - 2s 218us/step - loss: 1.4012 - accuracy: 0.8813
Epoch 8/100
7625/7625 [==============================] - 1s 186us/step - loss: 1.2054 - accuracy: 0.8825
Epoch 9/100
7625/7625 [==============================] - 1s 194us/step - loss: 1.0466 - accuracy: 0.8846
Epoch 10/100
7625/7625 [==============================] - 1s 191us/step - loss: 0.9144 - accuracy: 0.8893
Epoch 11/100
7625/7625 [==============================] - 1s 171us/step - loss: 0.8081 - accuracy: 0.8939
Epoch 12/100
7625/7625 [==============================] - 1s 167us/step - loss: 0.7171 - accuracy: 0.8995
Epoch 13/100
7625/7625 [==============================] - 1s 172us/step - loss: 0.6455 - accuracy: 0.9019
Epoch 14/100
7625/7625 [==============================] - 1s 166us/step - loss: 0.5822 - accuracy: 0.9079
Epoch 15/100
7625/7625 [==============================] - 1s 196us/step - loss: 0.5298 - accuracy: 0.9154
Epoch 16/100
7625/7625 [==============================] - 1s 

7625/7625 [==============================] - 2s 201us/step - loss: 0.1844 - accuracy: 0.9361
Epoch 85/100
7625/7625 [==============================] - 1s 188us/step - loss: 0.1855 - accuracy: 0.9378
Epoch 86/100
7625/7625 [==============================] - 1s 178us/step - loss: 0.1832 - accuracy: 0.9364
Epoch 87/100
7625/7625 [==============================] - 1s 176us/step - loss: 0.1846 - accuracy: 0.9377
Epoch 88/100
7625/7625 [==============================] - 1s 177us/step - loss: 0.1791 - accuracy: 0.9414
Epoch 89/100
7625/7625 [==============================] - 1s 187us/step - loss: 0.1778 - accuracy: 0.9395
Epoch 90/100
7625/7625 [==============================] - 1s 183us/step - loss: 0.1829 - accuracy: 0.9355
Epoch 91/100
7625/7625 [==============================] - 2s 204us/step - loss: 0.1836 - accuracy: 0.9361
Epoch 92/100
7625/7625 [==============================] - 2s 203us/step - loss: 0.1807 - accuracy: 0.9441
Epoch 93/100
7625/7625 [==============================] - 1

In [25]:
results['test_roc_auc'].mean()

0.7165542673707914

In [40]:
#Let's check how performence of the modell will look like without any regularization method
def DNN_model():
    model = Sequential()
    model.add(Dense(100, input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    #model.add(Dropout(0.3))
    model.add(Dense(100, kernel_initializer='normal', activation='relu',    kernel_regularizer=l1_l2(l1=0.0, l2=0.0)))
    #model.add(Dropout(0.3))
    model.add(Dense(100, kernel_initializer='normal', activation='relu', kernel_regularizer=l1_l2(l1=0.0, l2=0.0)))
    #model.add(Dropout(0.3))
    model.add(Dense(100, kernel_initializer='normal', activation='relu', kernel_regularizer=l1_l2(l1=0.0, l2=0.0)))
    #model.add(Dropout(0.3))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
    return model

dnn = KerasClassifier(build_fn=DNN_model, epochs=100, batch_size=1000, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
results = cross_validate(dnn, 
                        X=X, 
                        y=y, 
                        cv=kfold, 
                        scoring=['accuracy', 'balanced_accuracy', 'roc_auc', 'f1'], 
                        return_train_score=True, 
                        return_estimator=True)

Epoch 1/100
7624/7624 [==============================] - 4s 484us/step - loss: 0.4717 - accuracy: 0.8804
Epoch 2/100
7624/7624 [==============================] - 1s 151us/step - loss: 0.3421 - accuracy: 0.8805
Epoch 3/100
7624/7624 [==============================] - 1s 160us/step - loss: 0.3159 - accuracy: 0.8831
Epoch 4/100
7624/7624 [==============================] - 1s 162us/step - loss: 0.2788 - accuracy: 0.8869
Epoch 5/100
7624/7624 [==============================] - 1s 145us/step - loss: 0.2426 - accuracy: 0.8966
Epoch 6/100
7624/7624 [==============================] - 1s 180us/step - loss: 0.2002 - accuracy: 0.9129
Epoch 7/100
7624/7624 [==============================] - 1s 172us/step - loss: 0.1575 - accuracy: 0.9292
Epoch 8/100
7624/7624 [==============================] - 1s 156us/step - loss: 0.1120 - accuracy: 0.9357
Epoch 9/100
7624/7624 [==============================] - 1s 145us/step - loss: 0.0819 - accuracy: 0.9327
Epoch 10/100
7624/7624 [==============================]

7624/7624 [==============================] - 1s 135us/step - loss: 0.0674 - accuracy: 0.9175
Epoch 79/100
7624/7624 [==============================] - 1s 138us/step - loss: 0.0673 - accuracy: 0.9258
Epoch 80/100
7624/7624 [==============================] - 1s 141us/step - loss: 0.0673 - accuracy: 0.9187
Epoch 81/100
7624/7624 [==============================] - 1s 134us/step - loss: 0.0673 - accuracy: 0.9166
Epoch 82/100
7624/7624 [==============================] - 1s 117us/step - loss: 0.0673 - accuracy: 0.9189
Epoch 83/100
7624/7624 [==============================] - 1s 148us/step - loss: 0.0674 - accuracy: 0.9248
Epoch 84/100
7624/7624 [==============================] - 1s 136us/step - loss: 0.0675 - accuracy: 0.9191
Epoch 85/100
7624/7624 [==============================] - 1s 116us/step - loss: 0.0674 - accuracy: 0.9213
Epoch 86/100
7624/7624 [==============================] - 1s 148us/step - loss: 0.0673 - accuracy: 0.9218
Epoch 87/100
7624/7624 [==============================] - 1

7624/7624 [==============================] - 1s 121us/step - loss: 0.0790 - accuracy: 0.9244
Epoch 54/100
7624/7624 [==============================] - 1s 126us/step - loss: 0.0790 - accuracy: 0.9227
Epoch 55/100
7624/7624 [==============================] - 1s 127us/step - loss: 0.0792 - accuracy: 0.9180
Epoch 56/100
7624/7624 [==============================] - 1s 123us/step - loss: 0.0789 - accuracy: 0.9218
Epoch 57/100
7624/7624 [==============================] - 1s 133us/step - loss: 0.0792 - accuracy: 0.9225
Epoch 58/100
7624/7624 [==============================] - 1s 143us/step - loss: 0.0791 - accuracy: 0.9182
Epoch 59/100
7624/7624 [==============================] - 1s 133us/step - loss: 0.0790 - accuracy: 0.9212
Epoch 60/100
7624/7624 [==============================] - 1s 141us/step - loss: 0.0790 - accuracy: 0.9243
Epoch 61/100
7624/7624 [==============================] - 1s 123us/step - loss: 0.0791 - accuracy: 0.9217
Epoch 62/100
7624/7624 [==============================] - 1

7625/7625 [==============================] - 1s 147us/step - loss: 0.0961 - accuracy: 0.9158
Epoch 29/100
7625/7625 [==============================] - 1s 131us/step - loss: 0.0965 - accuracy: 0.9222
Epoch 30/100
7625/7625 [==============================] - 1s 135us/step - loss: 0.0958 - accuracy: 0.9203
Epoch 31/100
7625/7625 [==============================] - 1s 116us/step - loss: 0.0959 - accuracy: 0.9197
Epoch 32/100
7625/7625 [==============================] - 1s 131us/step - loss: 0.0955 - accuracy: 0.9205
Epoch 33/100
7625/7625 [==============================] - 1s 145us/step - loss: 0.0953 - accuracy: 0.9175
Epoch 34/100
7625/7625 [==============================] - 1s 150us/step - loss: 0.0955 - accuracy: 0.9209
Epoch 35/100
7625/7625 [==============================] - 1s 134us/step - loss: 0.0953 - accuracy: 0.9204
Epoch 36/100
7625/7625 [==============================] - 1s 160us/step - loss: 0.0954 - accuracy: 0.9186
Epoch 37/100
7625/7625 [==============================] - 1

7625/7625 [==============================] - 1s 171us/step - loss: 0.3167 - accuracy: 0.8851
Epoch 4/100
7625/7625 [==============================] - 1s 169us/step - loss: 0.2736 - accuracy: 0.8918
Epoch 5/100
7625/7625 [==============================] - 1s 170us/step - loss: 0.2308 - accuracy: 0.9011
Epoch 6/100
7625/7625 [==============================] - 1s 161us/step - loss: 0.1878 - accuracy: 0.9175
Epoch 7/100
7625/7625 [==============================] - 1s 165us/step - loss: 0.1615 - accuracy: 0.9225
Epoch 8/100
7625/7625 [==============================] - 1s 166us/step - loss: 0.1286 - accuracy: 0.9410
Epoch 9/100
7625/7625 [==============================] - 1s 179us/step - loss: 0.0961 - accuracy: 0.9437
Epoch 10/100
7625/7625 [==============================] - 1s 182us/step - loss: 0.0709 - accuracy: 0.9420
Epoch 11/100
7625/7625 [==============================] - 1s 185us/step - loss: 0.0577 - accuracy: 0.9346
Epoch 12/100
7625/7625 [==============================] - 1s 165u

...

7625/7625 [==============================] - 2s 200us/step - loss: 0.0722 - accuracy: 0.9294
Epoch 83/100
7625/7625 [==============================] - 2s 215us/step - loss: 0.0723 - accuracy: 0.9300
Epoch 84/100
7625/7625 [==============================] - 1s 187us/step - loss: 0.0720 - accuracy: 0.9258
Epoch 85/100
7625/7625 [==============================] - 1s 175us/step - loss: 0.0723 - accuracy: 0.9262
Epoch 86/100
7625/7625 [==============================] - 2s 213us/step - loss: 0.0720 - accuracy: 0.9267
Epoch 87/100
7625/7625 [==============================] - 1s 176us/step - loss: 0.0720 - accuracy: 0.9264
Epoch 88/100
7625/7625 [==============================] - 1s 175us/step - loss: 0.0722 - accuracy: 0.9294
Epoch 89/100
7625/7625 [==============================] - 1s 168us/step - loss: 0.0723 - accuracy: 0.9260
Epoch 90/100
7625/7625 [==============================] - 1s 186us/step - loss: 0.0724 - accuracy: 0.9336
Epoch 91/100
7625/7625 [==============================] - 2

7625/7625 [==============================] - 2s 210us/step - loss: 0.0697 - accuracy: 0.9304
Epoch 58/100
7625/7625 [==============================] - 2s 198us/step - loss: 0.0694 - accuracy: 0.9221
Epoch 59/100
7625/7625 [==============================] - 2s 218us/step - loss: 0.0693 - accuracy: 0.9234
Epoch 60/100
7625/7625 [==============================] - 1s 180us/step - loss: 0.0693 - accuracy: 0.9317
Epoch 61/100
7625/7625 [==============================] - 1s 138us/step - loss: 0.0695 - accuracy: 0.9237
Epoch 62/100
7625/7625 [==============================] - 1s 178us/step - loss: 0.0696 - accuracy: 0.9252
Epoch 63/100
7625/7625 [==============================] - 1s 195us/step - loss: 0.0692 - accuracy: 0.9255
Epoch 64/100
7625/7625 [==============================] - 1s 172us/step - loss: 0.0694 - accuracy: 0.9238
Epoch 65/100
7625/7625 [==============================] - 2s 237us/step - loss: 0.0692 - accuracy: 0.9246
Epoch 66/100
7625/7625 [==============================] - 1

7625/7625 [==============================] - 1s 181us/step - loss: 0.0624 - accuracy: 0.9334
Epoch 33/100
7625/7625 [==============================] - 1s 169us/step - loss: 0.0619 - accuracy: 0.9290
Epoch 34/100
7625/7625 [==============================] - 1s 159us/step - loss: 0.0624 - accuracy: 0.9266
Epoch 35/100
7625/7625 [==============================] - 1s 160us/step - loss: 0.0616 - accuracy: 0.9273
Epoch 36/100
7625/7625 [==============================] - 1s 141us/step - loss: 0.0619 - accuracy: 0.9321
Epoch 37/100
7625/7625 [==============================] - 1s 165us/step - loss: 0.0614 - accuracy: 0.9275
Epoch 38/100
7625/7625 [==============================] - 1s 148us/step - loss: 0.0613 - accuracy: 0.9249
Epoch 39/100
7625/7625 [==============================] - 1s 143us/step - loss: 0.0611 - accuracy: 0.9325
Epoch 40/100
7625/7625 [==============================] - 1s 187us/step - loss: 0.0612 - accuracy: 0.9309
Epoch 41/100
7625/7625 [==============================] - 1

7625/7625 [==============================] - 1s 137us/step - loss: 0.1530 - accuracy: 0.9318
Epoch 8/100
7625/7625 [==============================] - 1s 138us/step - loss: 0.1109 - accuracy: 0.9360
Epoch 9/100
7625/7625 [==============================] - 1s 172us/step - loss: 0.0802 - accuracy: 0.9382
Epoch 10/100
7625/7625 [==============================] - 1s 156us/step - loss: 0.0678 - accuracy: 0.9268
Epoch 11/100
7625/7625 [==============================] - 1s 128us/step - loss: 0.0648 - accuracy: 0.9288
Epoch 12/100
7625/7625 [==============================] - 1s 127us/step - loss: 0.0625 - accuracy: 0.9196
Epoch 13/100
7625/7625 [==============================] - 1s 120us/step - loss: 0.0661 - accuracy: 0.9258
Epoch 14/100
7625/7625 [==============================] - 1s 159us/step - loss: 0.0651 - accuracy: 0.9159
Epoch 15/100
7625/7625 [==============================] - 1s 133us/step - loss: 0.0674 - accuracy: 0.9267
Epoch 16/100
7625/7625 [==============================] - 1s 

7625/7625 [==============================] - 1s 159us/step - loss: 0.0725 - accuracy: 0.9258
Epoch 85/100
7625/7625 [==============================] - 1s 156us/step - loss: 0.0724 - accuracy: 0.9255
Epoch 86/100
7625/7625 [==============================] - 1s 136us/step - loss: 0.0724 - accuracy: 0.9196
Epoch 87/100
7625/7625 [==============================] - 1s 188us/step - loss: 0.0725 - accuracy: 0.9251
Epoch 88/100
7625/7625 [==============================] - 1s 140us/step - loss: 0.0725 - accuracy: 0.9216
Epoch 89/100
7625/7625 [==============================] - 1s 149us/step - loss: 0.0725 - accuracy: 0.9260
Epoch 90/100
7625/7625 [==============================] - 2s 216us/step - loss: 0.0724 - accuracy: 0.9251
Epoch 91/100
7625/7625 [==============================] - 1s 147us/step - loss: 0.0723 - accuracy: 0.9212
Epoch 92/100
7625/7625 [==============================] - 1s 138us/step - loss: 0.0723 - accuracy: 0.9201
Epoch 93/100
7625/7625 [==============================] - 1

In [41]:
results['test_roc_auc'].mean()

0.6902966779459122

In [37]:
#Compare with a Linear model
estimator = linear_model.LogisticRegression(max_iter=1000)
results = cross_validate(estimator, 
                                 X, 
                                 y.values.ravel(), 
                                 cv=kfold, 
                                 scoring=['accuracy', 'roc_auc', 'f1'], 
                                 return_train_score=True, 
                                 return_estimator=True)

In [39]:
results['test_roc_auc'].mean()

0.736176306202067

Questions and suggestions are highly welcome! adam.paclawski@uj.edu.pl